In [41]:
import numpy as np
import sys
from tqdm import tqdm

nDimensions = 3
charge = 2
stepLength = 1.5
nParticles = 2
# step length and square inverse
h = 0.001
h2 = 1000000
alpha = 1
nCycles = 1000000
thermalization = 1000

def local_energy(r):
    r_plus = np.zeros((nParticles, nDimensions))
    r_minus = np.zeros((nParticles, nDimensions))
    r_plus[:] = r
    r_minus[:] = r
    wave_function_minus = 0
    wave_function_plus = 0
    wave_function_current = wave_function(r)
    
    # Kinetic energy, brute force derivations
    kinetic_energy = 0
    for i in range(nParticles):
        for j in range(nDimensions):
            r_plus[i,j] += h
            r_minus[i,j] -= h
            wave_function_minus = wave_function(r_minus)
            wave_function_plus = wave_function(r_plus)
            kinetic_energy -= (wave_function_minus + wave_function_plus - 2 * wave_function_current)
            r_plus[i,j] = r[i,j]
            r_minus[i,j] = r[i,j]
    kinetic_energy = 0.5 * h2 * kinetic_energy / wave_function_current
    
    # Potential energy
    potential_energy = 0
    r_single_particle = 0
    for i in range(nParticles):
        r_single_particle = 0
        for j in range(nDimensions):
            r_single_particle += r[i,j] * r[i,j]
        potential_energy -= charge / np.sqrt(r_single_particle)
    
    # Contribution from electron-electron potential
    r12 = 0
    for i in range(nParticles):
        for j in range(i + 1, nParticles):
            r12 = 0
            for k in range(nDimensions):
                r12 += (r[i,k] - r[j,k]) * (r[i,k] - r[j,k])
            potential_energy += 1 / np.sqrt(r12)
    
    return kinetic_energy + potential_energy


def wave_function(r):
    argument = 0
    for i in range(nParticles):
        r_single_particle = 0
        for j in range(nDimensions):
            r_single_particle += r[i,j] * r[i,j]
        argument += np.sqrt(r_single_particle)
    return np.exp(-argument * alpha)

def grad_wave_func(r):
    grad_wave_func = np.zeros((nParticles,nDimensions))
    wave_func_old = wave_function(r)
    for i in range(nParticles):
        for j in range(nDimensions):
            r_new = r
            r_new[i,j] += h
            grad_wave_func[i,j] = (wave_function(r_new)-wave_func_old)/h
    return grad_wave_func

def solve_VMC(rInit):
    rOld = np.zeros((nParticles, nDimensions))
    rNew = np.zeros((nParticles, nDimensions))
    waveFunctionOld = 0
    waveFunctionNew = 0
    energySum = 0
    energySquaredSum = 0
    deltaE = 0
    prob = 1.
    # initial trial positions
    rOld = rInit

    rNew = rOld
    # loop over Monte Carlo cycles
    accept = 0
    for cycle in tqdm(range(nCycles), file=sys.stdout, desc='VMC Cycles'):
        prob = 1.
        # Store the current value of the wave function
        waveFunctionOld = wave_function(rOld)
        #direction = 2 * grad_wave_func(rOld)/waveFunctionOld * np.random.rand()
        direction = (np.random.rand(nParticles, nDimensions)-0.5)
        # New position to test
        for i in range(nParticles):
            for j in range(nDimensions):
                rNew[i,j] = rOld[i,j] + stepLength * direction[i,j]
            # Recalculate the value of the wave function
            waveFunctionNew = wave_function(rNew)
            # Check for step acceptance (if yes, update position, if no, reset position)
            if np.random.rand() <= np.float_power(waveFunctionNew/waveFunctionOld, 2):
                for j in range(nDimensions):
                    rOld[i,j] = rNew[i,j]
                waveFunctionOld = waveFunctionNew
                accept += 1
            else:
                for j in range(nDimensions):
                    rNew[i,j] = rOld[i,j]
            
            ### AVOID division error???????????????????????????????????????????????????????
            if wave_function(rOld) <= 1E-100:
                tqdm.write("triggered")
                rOld = stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)
                rNew = rOld + stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)

            # update energies
            if cycle > thermalization:
                deltaE = local_energy(rNew)
                energySum += deltaE
                energySquaredSum += deltaE*deltaE
        if cycle % 20000 == 0: tqdm.write("energy:"+str(round(energySum/((cycle+1) * nParticles)*2*13.6,2))+" (eV) ; acceptance rate = "+str(round(accept/(cycle+1)/nParticles * 100,2)))
    energy = energySum/(nCycles * nParticles)*2*13.6
    energySquared = energySquaredSum/(nCycles * nParticles)
    # print(f"Energy: {energy*2*13.6}(eV), Variance: {energySquared-energy**2}")
    return energy


In [42]:
solve_VMC(stepLength * (np.random.rand(nParticles, nDimensions) - 0.5))

energy:0.0 (eV) ; acceptance rate = 0.0                
energy:-26.45 (eV) ; acceptance rate = 72.85                         
triggered                                                            
energy:-44.85 (eV) ; acceptance rate = 69.79                         
energy:-51.51 (eV) ; acceptance rate = 68.64                         
energy:-54.98 (eV) ; acceptance rate = 68.18                         
energy:-56.73 (eV) ; acceptance rate = 67.97                         
energy:-58.28 (eV) ; acceptance rate = 67.68                          
energy:-59.17 (eV) ; acceptance rate = 67.45                          
energy:-59.72 (eV) ; acceptance rate = 67.31                          
energy:-60.16 (eV) ; acceptance rate = 67.26                          
energy:-60.53 (eV) ; acceptance rate = 67.24                          
energy:-60.86 (eV) ; acceptance rate = 67.18                          
energy:-61.05 (eV) ; acceptance rate = 67.15                          
energy:-61.27 (eV) ; accept

-63.71656869108275

In [43]:
(-2*alpha**2+4*charge*alpha-5/4*alpha)*13.6

64.6

In [44]:
for i in range(nParticles):
    for j in range(nDimensions):
        rOld[i,j] = stepLength * (np.random.rand() - 0.5)

NameError: name 'rOld' is not defined

In [ ]:
rOld